In [0]:
#%pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import *
from pyspark.ml.regression import LinearRegression , DecisionTreeRegressor, RandomForestRegressor, GBTRegressionModel, GBTRegressor

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.linalg import Vectors

from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.context import SparkContext
from pyspark.mllib.evaluation import MulticlassMetrics


In [0]:
#%pyspark
#PYSPARK_CLI = True
#if PYSPARK_CLI:
	#sc = SparkContext.getOrCreate()
	#spark = SparkSession(sc)

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2016_CaliforniaStateUniversity.csv"
#file_location = "/user/jdang8/2016_CaliforniaStateUniversity.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
#%pyspark
#df.printSchema()

In [0]:
# Create a view or table

temp_table_name = "2016_CaliforniaStateUniversity_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:

dftemp=df.select('EmployerName','Position','TotalWages', 'RegularPay', 'MaxPositionSalary', 'MinPositionSalary')
dftemp.show()

+-------------------+--------------------+----------+----------+-----------------+-----------------+
|       EmployerName|            Position|TotalWages|RegularPay|MaxPositionSalary|MinPositionSalary|
+-------------------+--------------------+----------+----------+-----------------+-----------------+
|Chancellor's Office|Accountant I, Ran...|   40945.9|  40315.14|          76152.0|          43968.0|
|Chancellor's Office|Accountant I, Ran...|  50238.35|  50238.35|          76152.0|          43968.0|
|Chancellor's Office|Accountant II, Ra...|   54546.6|   54546.6|          86460.0|          49932.0|
|Chancellor's Office|Accountant II, Ra...|  48541.86|  48341.86|          86460.0|          49932.0|
|Chancellor's Office|Accountant II, Ra...|  47879.45|  47957.15|          86460.0|          49932.0|
|Chancellor's Office|Accountant II, Ra...|   69480.6|   69480.6|          86460.0|          49932.0|
|Chancellor's Office|Accountant II, Ra...|   59295.0|   58745.0|          86460.0|         

In [0]:
%sql
select * from `2016_CaliforniaStateUniversity_csv` 

In [0]:
df.take(10)

Out[8]: [Row(Year=2016, EmployerType='California State University', EmployerName="Chancellor's Office", DepartmentOrSubdivision=None, Position='Accountant I, Range 1', ElectedOfficial=False, Judicial=False, OtherPositions=None, MinPositionSalary=43968.0, MaxPositionSalary=76152.0, ReportedBaseWage=None, RegularPay=40315.14, OvertimePay=130.76, LumpSumPay=0.0, OtherPay=500.0, TotalWages=40945.9, DefinedBenefitPlanContribution=10449.35, EmployeesRetirementCostCovered=0.0, DeferredCompensationPlan=0, HealthDentalVision=7762.92, TotalRetirementAndHealthContribution=18212.27, PensionFormula='2.00% @ 62', EmployerURL='http://www.calstate.edu/HRS/', EmployerPopulation=None, LastUpdatedDate='07/25/2017', EmployerCounty='Los Angeles', SpecialDistrictActivities=None, IncludesUnfundedLiability=None, SpecialDistrictType=None),
 Row(Year=2016, EmployerType='California State University', EmployerName="Chancellor's Office", DepartmentOrSubdivision=None, Position='Accountant I, Range 1', ElectedOffici

In [0]:
df_sql = spark.sql("select * from 2016_CaliforniaStateUniversity_csv LIMIT 10")
display(df_sql) 

Year,EmployerType,EmployerName,DepartmentOrSubdivision,Position,ElectedOfficial,Judicial,OtherPositions,MinPositionSalary,MaxPositionSalary,ReportedBaseWage,RegularPay,OvertimePay,LumpSumPay,OtherPay,TotalWages,DefinedBenefitPlanContribution,EmployeesRetirementCostCovered,DeferredCompensationPlan,HealthDentalVision,TotalRetirementAndHealthContribution,PensionFormula,EmployerURL,EmployerPopulation,LastUpdatedDate,EmployerCounty,SpecialDistrictActivities,IncludesUnfundedLiability,SpecialDistrictType
2016,California State University,Chancellor's Office,null,"Accountant I, Range 1",false,false,null,43968.0,76152.0,null,40315.14,130.76,0.0,500.0,40945.9,10449.35,0.0,0,7762.92,18212.27,2.00% @ 62,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant I, Range 1",false,false,null,43968.0,76152.0,null,50238.35,0.0,0.0,0.0,50238.35,13004.84,0.0,0,22866.32,35871.16,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,54546.6,0.0,0.0,0.0,54546.6,14153.64,0.0,0,9113.88,23267.52,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,48341.86,0.0,0.0,200.0,48541.86,12521.35,0.0,0,8348.04,20869.39,2.00% @ 62,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,47957.15,0.0,0.0,-77.7,47879.45,12498.92,0.0,0,12894.58,25393.5,2.00% @ 62,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,69480.6,0.0,0.0,0.0,69480.6,18166.73,0.0,0,9310.73,27477.46,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,58745.0,0.0,0.0,550.0,59295.0,15204.44,0.0,0,9091.08,24295.52,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,57039.93,0.0,0.0,0.0,57039.93,15548.23,0.0,0,23070.12,38618.35,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,64215.0,0.0,0.0,0.0,64215.0,16616.8,0.0,0,9003.12,25619.92,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null
2016,California State University,Chancellor's Office,null,"Accountant II, Range 1",false,false,null,49932.0,86460.0,null,61477.0,1563.81,0.0,1025.0,64065.81,15903.69,0.0,0,17217.75,33121.44,2.00% @ 55,http://www.calstate.edu/HRS/,null,07/25/2017,Los Angeles,null,null,null


In [0]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- EmployerType: string (nullable = true)
 |-- EmployerName: string (nullable = true)
 |-- DepartmentOrSubdivision: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- ElectedOfficial: boolean (nullable = true)
 |-- Judicial: boolean (nullable = true)
 |-- OtherPositions: string (nullable = true)
 |-- MinPositionSalary: double (nullable = true)
 |-- MaxPositionSalary: double (nullable = true)
 |-- ReportedBaseWage: string (nullable = true)
 |-- RegularPay: double (nullable = true)
 |-- OvertimePay: double (nullable = true)
 |-- LumpSumPay: double (nullable = true)
 |-- OtherPay: double (nullable = true)
 |-- TotalWages: double (nullable = true)
 |-- DefinedBenefitPlanContribution: double (nullable = true)
 |-- EmployeesRetirementCostCovered: double (nullable = true)
 |-- DeferredCompensationPlan: integer (nullable = true)
 |-- HealthDentalVision: double (nullable = true)
 |-- TotalRetirementAndHealthContribution: double (nul

In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- EmployerType: string (nullable = true)
 |-- EmployerName: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- MinPositionSalary: double (nullable = true)
 |-- MaxPositionSalary: double (nullable = true)
 |-- ReportedBaseWage: string (nullable = true)
 |-- RegularPay: double (nullable = true)
 |-- OvertimePay: double (nullable = true)
 |-- LumpSumPay: double (nullable = true)
 |-- OtherPay: double (nullable = true)
 |-- TotalWages: double (nullable = true)
 |-- LastUpdatedDate: string (nullable = true)
 |-- EmployerCounty: string (nullable = true)



In [0]:
display(df2)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|Year|        EmployerType|       EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|
+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|          76152.0|              NA|  40315.14|     130.76|       0.0|   500.0|   40945.9|     07/25/2017|   Los Angeles|
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|          76152.0|              NA|  50238.35|        0.0|       0.0|     0.0|  50238.

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

Training Rows: 83636 Testing Rows: 35839


In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|Year|        EmployerType|       EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|               wages|
+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|          76152.0|              NA|  50238.35|        0.0|       0.0|     0.0|  50238.35|     07/25/2017|   Los Angeles|[50238.35,50238.3...|
|2016|California State ...|Chancellor's Office|Accountant II, Ra...|          49932.0|  

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

,trueWages,importance
0,RegularPay,0.432901
1,TotalWages,0.391605
2,MaxPositionSalary,0.113492
3,MinPositionSalary,0.062002


In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[40315.14,40945.9...| 42675.67352589068|   40945.9|
|[47957.15,47879.4...| 50158.90193690494|  47879.45|
|[48341.86,48541.8...| 50158.90193690494|  48541.86|
|[58745.0,59295.0,...|  60686.7833583932|   59295.0|
|[64215.0,64215.0,...| 67580.24474807322|   64215.0|
|[6632.89,6632.89,...|7499.1676493274335|   6632.89|
|[45790.58,45790.5...|48096.024589464316|  45790.58|
|[51496.0,51496.0,...|50865.110398841105|   51496.0|
|[11227.64,12711.8...|12151.372670268407|  12711.81|
|[35850.34,38059.2...| 37138.18148513187|  38059.29|
|[40408.73,41716.0...| 42675.67352589068|  41716.09|
|[47175.19,48282.3...| 48785.54053368179|  48282.33|
|[51388.0,51963.0,...|50651.724568297985|   51963.0|
|[53011.0,53011.0,...|50651.724568297985|   53011.0|
|[62856.0,62856.0,...| 64678.02386004045|   62856.0|
|[55533.16,56707.6...|60000.334284092256|  567

In [0]:
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

R Squared (R2) on test data = 0.975241
RMSE: 5681.265627


#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|Year|        EmployerType|       EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|
+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|          76152.0|              NA|  40315.14|     130.76|       0.0|   500.0|   40945.9|     07/25/2017|   Los Angeles|
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|          76152.0|              NA|  50238.35|        0.0|       0.0|     0.0|  50238.

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

Training Rows: 83596 Testing Rows: 35879


In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|Year|        EmployerType|       EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|               wages|
+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|          76152.0|              NA|  40315.14|     130.76|       0.0|   500.0|   40945.9|     07/25/2017|   Los Angeles|[40315.14,40945.9...|
|2016|California State ...|Chancellor's Office|Accountant I, Ran...|          43968.0|  

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#@%pyspark
test.show()


+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|Year|        EmployerType|       EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|
+----+--------------------+-------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|2016|California State ...|Chancellor's Office|Accountant II, Ra...|          49932.0|          86460.0|              NA|   28936.9|        0.0|       0.0| -133.82|  28803.08|     07/25/2017|   Los Angeles|
|2016|California State ...|Chancellor's Office|Accountant II, Ra...|          49932.0|          86460.0|              NA|   54190.0|        0.0|       0.0|   475.0|   54665

In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[28936.9,28803.08...|29353.469021777604|  28803.08|
|[54190.0,54665.0,...| 54078.12773205787|   54665.0|
|[61228.0,61978.0,...| 61987.73213002078|   61978.0|
|[47244.9,47844.9,...| 46760.54239824802|   47844.9|
|[26220.65,30364.4...|29353.469021777604|  30364.41|
|[35850.34,38059.2...|35572.711212348164|  38059.29|
|[46319.28,46319.2...|46818.754601800734|  46319.28|
|[47175.19,48282.3...|46818.754601800734|  48282.33|
|[55318.24,55668.2...| 53944.74232071551|  55668.24|
|[56523.56,56523.5...| 58671.61544281146|  56523.56|
|[64614.16,64739.1...| 67366.79954338231|  64739.16|
|[58020.0,58020.0,...| 58881.05845613305|   58020.0|
|[61388.6,61423.19...| 62197.17514334236|  61423.19|
|[64423.0,64423.0,...| 67698.41802489058|   64423.0|
|[28828.5,28828.5,...|29380.126635081873|   28828.5|
|[3662.5,3662.5,60...|3741.2317305246997|    3

In [0]:
#%pyspark
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[28936.9,28803.08...|29353.469021777604|  28803.08|
|[54190.0,54665.0,...| 54078.12773205787|   54665.0|
|[61228.0,61978.0,...| 61987.73213002078|   61978.0|
|[47244.9,47844.9,...| 46760.54239824802|   47844.9|
|[26220.65,30364.4...|29353.469021777604|  30364.41|
|[35850.34,38059.2...|35572.711212348164|  38059.29|
|[46319.28,46319.2...|46818.754601800734|  46319.28|
|[47175.19,48282.3...|46818.754601800734|  48282.33|
|[55318.24,55668.2...| 53944.74232071551|  55668.24|
|[56523.56,56523.5...| 58671.61544281146|  56523.56|
|[64614.16,64739.1...| 67366.79954338231|  64739.16|
|[58020.0,58020.0,...| 58881.05845613305|   58020.0|
|[61388.6,61423.19...| 62197.17514334236|  61423.19|
|[64423.0,64423.0,...| 67698.41802489058|   64423.0|
|[28828.5,28828.5,...|29380.126635081873|   28828.5|
|[3662.5,3662.5,60...|3741.2317305246997|    3

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

R Squared (R2) on test data = 0.985298
RMSE: 4341.954083


In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[28936.9,28803.08...|29353.469021777604|  28803.08|
|[54190.0,54665.0,...| 54078.12773205787|   54665.0|
|[61228.0,61978.0,...| 61987.73213002078|   61978.0|
|[47244.9,47844.9,...| 46760.54239824802|   47844.9|
|[26220.65,30364.4...|29353.469021777604|  30364.41|
|[35850.34,38059.2...|35572.711212348164|  38059.29|
|[46319.28,46319.2...|46818.754601800734|  46319.28|
|[47175.19,48282.3...|46818.754601800734|  48282.33|
|[55318.24,55668.2...| 53944.74232071551|  55668.24|
|[56523.56,56523.5...| 58671.61544281146|  56523.56|
|[64614.16,64739.1...| 67366.79954338231|  64739.16|
|[58020.0,58020.0,...| 58881.05845613305|   58020.0|
|[61388.6,61423.19...| 62197.17514334236|  61423.19|
|[64423.0,64423.0,...| 67698.41802489058|   64423.0|
|[28828.5,28828.5,...|29380.126635081873|   28828.5|
|[3662.5,3662.5,60...|3741.2317305246997|    3

In [0]:
####CODE BREAK for 2015

#CSU2015

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2015_CaliforniaStateUniversity.csv"
#file_location = "/user/jdang8/2015_CaliforniaStateUniversity.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2015_CaliforniaStateUniversity_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

root



In [0]:
display(df2)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

++
||
++
++



In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

Training Rows: 0 Testing Rows: 0


In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-1679082689279821> in <module>
----> 1 training = assembler.transform(train)
      2 training.show()

/databricks/spark/python/pyspark/ml/base.py in transform(self, dataset, params)
    215                 return self.copy(params)._transform(dataset)
    216             else:
--> 217                 return self._transform(dataset)
    218         else:
    219             raise TypeError("Params must be a param map but got %s." % type(params))

/databricks/spark/python/pyspark/ml/wrapper.py in _transform(self, dataset)
    348     def _transform(self, dataset):
    349         self._transfer_params_to_java()
--> 350         return DataFrame(self._java_obj.transform(dataset._jdf), dataset.sql_ctx)
    351 
    352 

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303  

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

In [0]:
#GBT 2015


#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#### Code Break for 2017

#CSU 2017

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2017_CaliforniaStateUniversity.csv"
#file_location = "/user/jdang8/2017_CaliforniaStateUniversity.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2017_CaliforniaStateUniversity_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#@%pyspark
test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
# Code Break for 2018

#CSU 2018

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2018_CaliforniaStateUniversity.csv"
#file_location = "/user/jdang8/2018_CaliforniaStateUniversity.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2018_CaliforniaStateUniversity_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#@%pyspark
test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
### Code Break for 2019

#CSU 2019

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2019_CaliforniaStateUniversity.csv"
#file_location = "/user/jdang8/2019_CaliforniaStateUniversity.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2019_CaliforniaStateUniversity_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
### Code break for 2020

#CSU 2020

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2020_CaliforniaStateUniversity.csv"
#file_location = "/user/jdang8/2020_CaliforniaStateUniversity.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2020_CaliforniaStateUniversity_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)
#%pyspark
model = cv.fit(train)
#@%pyspark
test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()